In [1]:
import os

In [4]:
%pwd

'd:\\Projects\\NLP\\Text-Summarizer-Project'

In [3]:
os.chdir('../')

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    SOURCE_URL : str
    local_data_file : Path
    unzip_dir : Path


In [11]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        #create a artifacts folder\
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            SOURCE_URL= config.SOURCE_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [13]:
#update the components

import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [15]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.SOURCE_URL,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} download ! with the following info : \n{headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r')as zip_ref:
            zip_ref.extractall(unzip_path)

            

In [18]:
#create the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e


[2023-09-21 09:54:22,306: INFO: common: yaml file : config\config.yaml loaded sucessfully]
[2023-09-21 09:54:22,313: INFO: common: yaml file : params.yaml loaded sucessfully]
[2023-09-21 09:54:22,586: INFO: common: created directory at : artifacts]
[2023-09-21 09:54:22,592: INFO: common: created directory at : artifacts/data_ingestion]
[2023-09-21 09:57:07,750: INFO: 405531993: artifacts/data_ingestion/data.zip download ! with the following info : 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 139E:24C5D8:FD16:1F3B7:650BC585
Accept-Ranges: bytes
Date: Thu, 21 Sep 2023 04:24:38 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830029-CCU
X-Cac